```{contents}
```
## Authentication

**Authentication** in LangGraph governs **who is allowed to invoke graphs, access state, use tools, and interact with agents**.
While LangGraph itself is an orchestration engine, **production-grade authentication is enforced at the application and runtime layers** surrounding the graph.

LangGraph’s design allows authentication information to be **injected into execution context**, propagated through **state**, and enforced by **nodes, tools, and policies**.

---

### **1. Where Authentication Lives in a LangGraph System**

```
Client → API Gateway → Auth Layer → LangGraph Runtime → Graph Nodes → Tools / Models
```

| Layer             | Responsibility                       |
| ----------------- | ------------------------------------ |
| Client            | Sends identity (token, cookie, cert) |
| API Gateway       | Verifies identity                    |
| Auth Layer        | Issues claims / permissions          |
| LangGraph Runtime | Enforces access inside workflows     |
| Nodes & Tools     | Enforce fine-grained permissions     |

---

### **2. Authentication Data Flow**

1. User logs in
2. Receives **access token** (JWT, OAuth2, session cookie)
3. Token sent with each request
4. API verifies token
5. Claims injected into **LangGraph execution context**
6. Graph nodes make authorization decisions

---

### **3. Execution Context with Identity**

```python
from langgraph.graph import StateGraph

def entry_node(state, config):
    user = config["user"]        # identity injected at runtime
    return {"user_id": user["id"], "role": user["role"]}
```

Graph invocation:

```python
graph.invoke(
    {"query": "Generate report"},
    config={
        "user": {
            "id": "u123",
            "role": "admin",
            "permissions": ["read", "write", "deploy"]
        }
    }
)
```

The **config object** becomes the **security context**.

---

### **4. State-Based Authorization**

Store identity and permissions in state:

```python
class State(TypedDict):
    user_id: str
    role: str
    permissions: list[str]
    result: str
```

Authorize inside nodes:

```python
def secure_node(state):
    if "deploy" not in state["permissions"]:
        raise PermissionError("Access denied")
    return {"result": "Deployment executed"}
```

---

### **5. Tool-Level Authentication Enforcement**

Wrap tools with permission checks:

```python
def secure_tool(state):
    if state["role"] != "admin":
        return {"error": "Unauthorized"}
    # execute tool
```

This ensures **no tool can be executed without proper identity**.

---

### **6. Common Authentication Models**

| Model           | Usage                     |
| --------------- | ------------------------- |
| JWT / OAuth2    | Web & API systems         |
| API Keys        | Service-to-service        |
| Session Cookies | Browser apps              |
| mTLS            | Zero-trust infrastructure |
| IAM Roles       | Cloud-native deployments  |

---

### **7. Multi-Tenant Authentication**

Store tenant identity in context:

```python
config = {
    "user": {"id": "u12", "tenant": "orgA", "role": "manager"}
}
```

Enforce isolation:

```python
def tenant_guard(state):
    if state["tenant"] != expected_tenant:
        raise SecurityError("Cross-tenant access")
```

---

### **8. Human-in-the-Loop Authentication**

Only authorized roles may approve:

```python
def approval_node(state):
    if state["role"] not in ["manager", "admin"]:
        raise PermissionError("Approval denied")
```

---

### **9. Production Architecture Example**

```
Client → Auth Service → API Gateway
                 ↓
            Identity Provider
                 ↓
LangGraph Runtime (enforces identity)
                 ↓
LLMs / Tools / Databases
```

---

### **10. Security Best Practices for LangGraph**

| Practice                                    | Purpose                      |
| ------------------------------------------- | ---------------------------- |
| Never trust node input                      | Prevent privilege escalation |
| Inject identity only at boundary            | Avoid tampering              |
| Validate permissions at each sensitive node | Defense in depth             |
| Log identity with every execution           | Auditability                 |
| Encrypt tokens at rest                      | Data protection              |
| Rotate keys                                 | Minimize breach impact       |

---

### **11. Why Authentication Matters in LangGraph**

LangGraph enables:

* Autonomous agents
* Tool execution
* Long-running workflows

Without strong authentication, these systems become **high-risk attack surfaces**.

Authentication transforms LangGraph from a prototype tool into a **secure enterprise orchestration platform**.


### Demonstration

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Secure State Definition
# ---------------------------
class State(TypedDict, total=False):
    _auth: dict
    user_id: str
    role: str
    permissions: list[str]
    result: str

# ---------------------------
# 2. Entry Node: Extract Identity
# ---------------------------
def entry_node(state):
    user = state.get("_auth")
    if not user:
        raise PermissionError("Missing authentication")

    return {
        "user_id": user["id"],
        "role": user["role"],
        "permissions": user["permissions"]
    }

# ---------------------------
# 3. Secure Business Node
# ---------------------------
def secure_action(state):
    if "deploy" not in state.get("permissions", []):
        raise PermissionError("Access denied: missing 'deploy' permission")
    return {"result": "Deployment executed successfully"}

# ---------------------------
# 4. Build Graph
# ---------------------------
builder = StateGraph(State)

builder.add_node("entry", entry_node)
builder.add_node("secure_action", secure_action)

builder.set_entry_point("entry")
builder.add_edge("entry", "secure_action")
builder.add_edge("secure_action", END)

graph = builder.compile()

# ---------------------------
# 5. Authorized Invocation
# ---------------------------
print(
    graph.invoke({
        "_auth": {
            "id": "u123",
            "role": "admin",
            "permissions": ["read", "write", "deploy"]
        }
    })
)

# ---------------------------
# 6. Unauthorized Invocation
# ---------------------------
try:
    graph.invoke({
        "_auth": {
            "id": "u999",
            "role": "viewer",
            "permissions": ["read"]
        }
    })
except Exception as e:
    print("SECURITY BLOCK:", e)


{'_auth': {'id': 'u123', 'role': 'admin', 'permissions': ['read', 'write', 'deploy']}, 'user_id': 'u123', 'role': 'admin', 'permissions': ['read', 'write', 'deploy'], 'result': 'Deployment executed successfully'}
SECURITY BLOCK: Access denied: missing 'deploy' permission
